In [1]:
import json
import shutil
import os
import re
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.scripts.export_kitti import KittiConverter
from nuscenes.utils.data_classes import LidarPointCloud
from pyquaternion import Quaternion
from tqdm import tqdm
from PIL import Image
import pcl
import cv2

from pykitti.raw import raw
from pykitti import utils
from nuscenes2kitti import nuscenes2kitti as nusc2kitti

In [2]:
datasets_root = f'/disk/ml/datasets/'
kitti_raw_root = f'{datasets_root}KITTI_Raw/'
kitti_object_calib_root = f'{datasets_root}KITTI/object/data/training/calib/'
nuscenes_root = f'{datasets_root}nuScenes/'
once_root = f'{datasets_root}ONCE/data_root/data/'
coda_root = f'/disk/ml/own_datasets/CODA/'

corner_case_file = f'{coda_root}corner_case.json'
kitti_mapping_file = f'{coda_root}kitti_mapping.json'
nuscenes_indices_file = f'{coda_root}nuscenes_indices.json'
nuscenes_poses_folder = f'/disk/vanishing_data/ju878/nusc_kitti_test/train/oxts/poses/'
once_poses_folder = f'/disk/vanishing_data/ju878/once_kitti_test/poses/'

dst_dir = f'/disk/vanishing_data/ju878/CODA_for_Finn_sequences/'
sequence_folder = f'{dst_dir}sequences/'
poses_folder= f'{dst_dir}poses/'

pattern = r'^\D*(\d\D*){6}$'

if not os.path.exists(sequence_folder):
        os.makedirs(sequence_folder)

if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)
        
        
with open(corner_case_file, 'r') as f:
    corner_case = json.load(f)

with open(kitti_mapping_file, 'r') as f:
    kitti_mapping = json.load(f)
    
with open(nuscenes_indices_file, 'r') as f:
    nuscenes_indices = json.load(f)


images = corner_case['images']

In [9]:
image_id = 1100
nusc_version = f'v1.0-trainval'
nusc_sensor_image = f'CAM_FRONT'
nusc_sensor_lidar = f'LIDAR_TOP'
num_images = 8

do_kitti = False
do_nuscenes = False
do_once = True

copy_images = True
copy_velodyne = False
copy_calib = False
copy_poses = False
copy_timestamps = False

In [15]:
nusc = NuScenes(version=nusc_version, dataroot=nuscenes_root, verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 60.348 seconds.
Reverse indexing ...
Done reverse indexing in 8.5 seconds.


In [16]:
kitti_conv = nusc2kitti.KittiConverter(poses_folder, nusc_sensor_image, nusc_sensor_lidar)

In [4]:
def read_calib(calib_path):
    data = {}
    with open(calib_path, 'r') as f:
        lines = filter(None, (line.rstrip() for line in f))
        for line in lines:
            if line == ' ':
                break
            else:
                key, value = line.split(':', 1)
                # The only non-float values in these files are dates, which
                # we don't care about anyway
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
    return data

In [5]:
def kitti_timestamp_to_ms(timestamp):
    parts = timestamp.split(' ')[1].replace('\n','').split(':')
    h = float(parts[0])
    m = float(parts[1])
    s = float(parts[2].split('.')[0])
    ms = float(parts[2].split('.')[1]) / 1000000
    
    new_times = (h * 60 * 60 * 1000) + (m * 60 * 1000) + (s * 1000) + ms
    
    return new_times

In [10]:
amount = 0
numbers = ''
numbers_vert = []
for image in tqdm(images):
    if image['id'] <= image_id:
        #image_id = image['id'] - 1
        #print(image['id'])
        id = image['id']
        file_name = image['file_name'].split('.')[0]
        prefix, name = file_name.split('_')
        
        id_str = '{:0>4}'.format(id)
        sequence_root = f'{sequence_folder}{id_str}/'
        
        
        if prefix == 'kitti' and do_kitti:
            file_name_ending = f'{file_name}.png'
            calib = read_calib(f'{kitti_object_calib_root}{name}.txt')
            image_2_folder = f'{sequence_root}image_2/'
            velodyne_folder = f'{sequence_root}velodyne/'
            
            raw_file_name = kitti_mapping[file_name_ending].replace('\n','').split(' ')
            raw_sequence_root = f'{kitti_raw_root}{raw_file_name[0]}/{raw_file_name[1]}/'
            raw_timestamp_file = f'{raw_sequence_root}velodyne_points/timestamps.txt'
            
            raw_image_number = int(raw_file_name[2])
            
            if raw_image_number >= num_images:
                  
                if not os.path.exists(sequence_root):
                    os.makedirs(sequence_root)
                
                if not os.path.exists(image_2_folder):
                    os.makedirs(image_2_folder)

                if not os.path.exists(velodyne_folder):
                    os.makedirs(velodyne_folder)
                    
                if copy_images or copy_velodyne:
                    for i in range(raw_image_number - num_images, raw_image_number + num_images + 1):

                        old_file_number = '{:0>10}'.format(i)
                        source_image = f'{raw_sequence_root}image_02/data/{old_file_number}.png'
                        source_lidar = f'{raw_sequence_root}velodyne_points/data/{old_file_number}.bin'

                        new_file_number = -raw_image_number + num_images + i
                        destination_image = f'{image_2_folder}{file_name}_{new_file_number}.png'
                        destination_lidar = f'{velodyne_folder}{file_name}_{new_file_number}.bin'

                        shutil.copy(source_image, destination_image)
                        shutil.copy(source_lidar, destination_lidar)
                        
                if copy_calib:
                    Tr_velo_to_cam = np.vstack((calib['Tr_velo_to_cam'].reshape(3, 4), [0, 0, 0, 1]))
                    R0_rect = np.eye(4)
                    R0_rect[0:3, 0:3] = np.reshape(calib['R0_rect'], (3, 3))
                    Tr = R0_rect.dot(Tr_velo_to_cam)
                    
                    kitti_transforms = dict()
                    kitti_transforms['P0'] = calib['P0'].reshape(3, 4)
                    kitti_transforms['P1'] = calib['P1'].reshape(3, 4)
                    kitti_transforms['P2'] = calib['P2'].reshape(3, 4)
                    kitti_transforms['P3'] = calib['P3'].reshape(3, 4)
                    #kitti_transforms['R0_rect'] = r0_rect.rotation_matrix  # Cameras are already rectified.
                    kitti_transforms['Tr'] = Tr[0:3][0:4]
                    #kitti_transforms['Tr_imu_to_velo'] = imu_to_velo_kitti
                    calib_path = os.path.join(sequence_root, 'calib.txt')
                    
                    with open(calib_path, "w") as f:
                        for (key, val) in kitti_transforms.items():
                            val = val.flatten()
                            val_str = '%.12e' % val[0]
                            for v in val[1:12]:
                                val_str += ' %.12e' % v
                            f.write('%s: %s\n' % (key, val_str))
                            
                if copy_poses:   
                    poses = []

                    with open(f'{raw_sequence_root}/poses.txt', 'r') as f:
                        lines = filter(None, (line.rstrip() for line in f))
                        for index, line in enumerate(lines):
                            if index >= raw_image_number - num_images and index <= raw_image_number + num_images:
                                poses.append(line)

                    pose_file = '{:0>4}'.format(id)
                    with open(f'{poses_folder}{pose_file}.txt', 'w') as f:
                        for pose in poses:
                            f.write('%s\n' % (pose))
                    with open(f'{sequence_root}poses.txt', 'w') as f:
                        for pose in poses:
                            f.write('%s\n' % (pose)) 
                            
                if copy_timestamps:
                    times = []
                    first_time = 0

                    with open(f'{raw_timestamp_file}', 'r') as f:
                        lines = filter(None, (line.rstrip() for line in f))
                        for index, line in enumerate(lines):
                            if index == 0:
                                first_time = kitti_timestamp_to_ms(line)
                            if index >= raw_image_number - num_images and index <= raw_image_number + num_images:
                                time = kitti_timestamp_to_ms(line)
                                time_diff = time - first_time
                                times.append(time_diff / 1000)
                                
                    times.sort()
                    with open(f'{sequence_root}times.txt', 'w') as f:
                        for time in times:
                            f.write('%s\n' % (time)) 
                
                amount=amount 
                numbers = numbers + f'{id},'
                numbers_vert.append(f'    - {id}')
            else:
                if os.path.exists(sequence_root):
                    os.rmdir(sequence_root)
                #print(id)
                amount += 1
                
                
                    
        elif prefix == 'nuscenes' and do_nuscenes:
            sample_tokens = []
            sample_names = []
            nuscenes_token = nuscenes_indices[image['file_name']]
            sample = nusc.get('sample', nuscenes_token)
            scene = nusc.get('scene', sample['scene_token'])
            scene_name = scene['name']
            
            prev = sample
            next = sample
            
            sample_tokens.append(sample['token'])
            sample_names.append(f'{file_name}_{num_images}')
            
            counter_prev = 0
            counter_next = 0
            
            while prev['prev'] != '':
                counter_prev += 1
                prev = nusc.get('sample', prev['prev'])  
            while next['next'] != '':
                counter_next += 1
                next = nusc.get('sample', next['next'])
            
            prev = sample
            next = sample
            
            if counter_prev >= num_images and counter_next >= num_images:   
                
                if not os.path.exists(sequence_root):
                    os.makedirs(sequence_root)
                
                if copy_images or copy_velodyne or copy_calib or copy_timestamps:
                    for i in range(1, num_images + 1):
                        prev = nusc.get('sample', prev['prev'])
                        sample_tokens.append(prev['token'])
                        sample_names.append(f'{file_name}_{num_images - i}')

                        next = nusc.get('sample', next['next'])
                        sample_tokens.append(next['token'])
                        sample_names.append(f'{file_name}_{num_images + i}')

                    kitti_conv.nuscenes_gt_to_kitti(sample_tokens, sample_names, sequence_root, nusc)
                
                if copy_poses:
                    poses = []
                    np_poses = []

                    with open(f'{nuscenes_poses_folder}{scene_name}.txt', 'r') as f:
                        lines = filter(None, (line.rstrip() for line in f))
                        for index, line in enumerate(lines):
                            if index >= counter_prev - num_images and index <= counter_prev + num_images:
                                poses.append(line)
                                np_poses.append(np.array([float(x) for x in line.split()]))
                                
                    pose_file = '{:0>4}'.format(id)
                    with open(f'{poses_folder}{pose_file}.txt', 'w') as f:
                        for pose in poses:
                            f.write('%s\n' % (pose)) 
                    with open(f'{sequence_root}poses.txt', 'w') as f:
                        for pose in poses:
                            f.write('%s\n' % (pose)) 
                            
                                
                amount = amount
                numbers = numbers + f'{id},'
                numbers_vert.append(f'    - {id}')
            else:
                if os.path.exists(sequence_root):
                    os.rmdir(sequence_root)
                #print(id)
                amount += 1
                
        elif re.match(pattern, prefix) and do_once:
            image_2_folder = f'{sequence_root}image_2/'
            velodyne_folder = f'{sequence_root}velodyne/'
            
            sequence, file = file_name.split('.')[0].split('_')
            once_sequence_root = f'{once_root}{sequence}/'
            once_image_root = f'{once_sequence_root}cam03/'
            once_lidar_root = f'{once_sequence_root}lidar_roof/'
            once_json_file = f'{once_sequence_root}{sequence}.json'
            
            with open(once_json_file, 'r') as f:
                once_json = json.load(f)
            
            calib = once_json['calib']
            calib = calib['cam03']
            
            cam_to_velo = np.array(calib['cam_to_velo'])
            velo_to_cam = np.linalg.inv(cam_to_velo)
            new_velo_to_cam = np.zeros_like(velo_to_cam)
            new_velo_to_cam[:, 1] = velo_to_cam[:, 0]
            new_velo_to_cam[:, 0] = -velo_to_cam[:, 1]
            new_velo_to_cam[:, 2] = velo_to_cam[:, 2]
            
            cam_intrinsic = np.array(calib['cam_intrinsic'])
            distortion = np.array(calib['distortion'])
            
            #new_cam_intrinsic = np.zeros_like(cam_intrinsic)
            #new_cam_intrinsic[1, :] = cam_intrinsic[0, :]
            #new_cam_intrinsic[0, :] = -cam_intrinsic[1, :]
            #new_cam_intrinsic[2, :] = cam_intrinsic[2, :]
            #new_cam_intrinsic_n, _ = cv2.getOptimalNewCameraMatrix(new_cam_intrinsic, distortion, (1920, 1020), alpha=0.0, newImgSize=(1920, 1020))
            #new_cam_intrinsic = np.hstack([new_cam_intrinsic_n, np.zeros((3, 1), dtype=np.float32)])
            
            cam_intrinsic_n, roi = cv2.getOptimalNewCameraMatrix(cam_intrinsic, distortion, (1920, 1020), alpha=0.0, newImgSize=(1920, 1020))
            cam_intrinsic = np.hstack([cam_intrinsic_n, np.zeros((3, 1), dtype=np.float32)]) #cam_intrinsic_n
            
            
            frames = once_json['frames']
            
            kitti_to_once_lidar = Quaternion(axis=(0, 0, 1), angle=np.pi * 3 / 2)
            kitti_to_once_lidar_inv = kitti_to_once_lidar.inverse
            
            velo_to_cam_kitti = np.dot(velo_to_cam, kitti_to_once_lidar.transformation_matrix)
            
            first_timestamp = frames[0]['frame_id']
            frames_len = len(frames)
            file_index = 0
            for index, frame in enumerate(frames):
                if frame['frame_id'] == file:
                    file_index = index
                    
            
            if file_index >= num_images and file_index <= frames_len - num_images - 1:
                
                if not os.path.exists(sequence_root):
                    os.makedirs(sequence_root)
                if not os.path.exists(image_2_folder):
                    os.makedirs(image_2_folder)
                if not os.path.exists(velodyne_folder):
                    os.makedirs(velodyne_folder)
                
                if copy_images or copy_velodyne:    
                    for index, frame_index in enumerate(range(file_index - num_images , file_index + num_images + 1)):
                        
                        once_file_name = frames[frame_index]['frame_id']
                        source_image = f'{once_image_root}{once_file_name}.jpg'
                        source_lidar = f'{once_lidar_root}{once_file_name}.bin'
                        
                        destination_image = f'{image_2_folder}{file_name}_{index}.png'
                        destination_lidar = f'{velodyne_folder}{file_name}_{index}.bin'
                        
                        img = cv2.imread(source_image)
                        dst = cv2.undistort(img, np.array(calib['cam_intrinsic']), distortion, None, cam_intrinsic_n)
                        x, y, w, h = roi
                        dst = dst[y:y+h, x:x+w]
                        cv2.imwrite(destination_image, dst)
                        #im = Image.open(source_image)
                        #cv2.undistort
                        #im.save(destination_image, "PNG")
                        
                        #shutil.copy(source_lidar, destination_lidar) 
                        
                        points = np.fromfile(source_lidar, dtype=np.float32).reshape((-1, 4))
                        new_points = np.zeros_like(points)
                        new_points[:, 1] = points[:, 0]
                        new_points[:, 0] = -points[:, 1]
                        new_points[:, 2] = points[:, 2]
                        #points[:3, :] = np.dot(kitti_to_once_lidar_inv.rotation_matrix, points[:3, :])  # In KITTI lidar frame.
                        with open(destination_lidar, "w") as lid_file:
                            new_points.tofile(lid_file)                       
                
                if copy_calib:
                    
                    kitti_transforms = dict()
                    kitti_transforms['P0'] = np.zeros((3, 4))  # Dummy values.
                    kitti_transforms['P1'] = np.zeros((3, 4))  # Dummy values.
                    kitti_transforms['P2'] = cam_intrinsic # cam_intrinsic @ cam_to_velo #np.hstack((cam_intrinsic, np.zeros((3, 1))))  # Left camera transform.
                    kitti_transforms['P3'] = np.zeros((3, 4))  # Dummy values.
                    #kitti_transforms['R0_rect'] = r0_rect.rotation_matrix  # Cameras are already rectified.
                    kitti_transforms['Tr'] = new_velo_to_cam[:3, :] #np.linalg.inv(cam_to_velo)[:3, :]
                    #kitti_transforms['Tr_imu_to_velo'] = imu_to_velo_kitti
                    calib_path = os.path.join(sequence_root, 'calib.txt')
                    with open(calib_path, "w") as calib_file:
                        for (key, val) in kitti_transforms.items():
                            val = val.flatten()
                            val_str = '%.12e' % val[0]
                            for v in val[1:]:
                                val_str += ' %.12e' % v
                            calib_file.write('%s: %s\n' % (key, val_str))
                            
                if copy_poses:
                    poses = []
                    np_poses = []

                    with open(f'{once_poses_folder}{sequence}.txt', 'r') as f:
                        lines = filter(None, (line.rstrip() for line in f))
                        for index, line in enumerate(lines):
                            if index >= file_index - num_images and index <= file_index + num_images:
                                poses.append(line)
                                np_poses.append(np.array([float(x) for x in line.split()]))
                    pose_file = '{:0>4}'.format(id)
                    with open(f'{poses_folder}{pose_file}.txt', 'w') as f:
                        for index, pose in enumerate(np_poses):
                            if index == 0:
                                f.write('1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0\n')
                            else:
                                pose = pose.flatten()
                                pose_str = '' #'%.12e' % (1 - (np_poses[0].flatten()[0]) - pose[0])
                                #print((np_poses[0].flatten()[0]) - pose[0])
                                for i, p in enumerate(pose):
                                    if i == 0:
                                        pose_str += '%.12e' % (p + (1 - np_poses[0].flatten()[i]))
                                    elif i == 5 or i == 10:
                                        #print(p + (1 - np_poses[0].flatten()[i]))
                                        pose_str += ' %.12e' % (p + (1 - np_poses[0].flatten()[i]))
                                    else:
                                        #print((np_poses[0].flatten()))
                                        pose_str += ' %.12e' % (p + (0 - np_poses[0].flatten()[i]))
                                f.write('%s\n' % (pose_str))
                            #f.write('%s\n' % (pose)) 
                    with open(f'{sequence_root}poses.txt', 'w') as f:
                        for pose in poses:
                            f.write('%s\n' % (pose))
                
                if copy_timestamps:
                    times = []
                    for index, frame_index in enumerate(range(file_index - num_images , file_index + num_images + 1)):
                        timestamp = frames[frame_index]['frame_id']
                        time_diff = float(timestamp) - float(first_timestamp)
                        times.append(time_diff / 1000)
                                
                    times.sort()
                    with open(f'{sequence_root}times.txt', 'w') as f:
                        for time in times:
                            f.write('%s\n' % (time)) 
                amount = amount
                numbers = numbers + f'{id},'
                numbers_vert.append(f'    - {id}')
            else:
                if os.path.exists(sequence_root):
                    os.rmdir(sequence_root)
                #print(id)
                amount += 1
print(amount)
print(numbers)
for number in numbers_vert:
    print(number)

    

  0%|          | 0/1500 [00:00<?, ?it/s]